In [ ]:
!pip install ratsnlp

## BPE 기반 토크나이저 만들기

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


In [ ]:
from Korpora import Korpora

In [ ]:
#bpe 수행대상 말뭉치 내려받고 전차리하기
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 69.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 73.6MB/s]


In [ ]:
import os

In [ ]:
#nsmc 리뷰를 텍스트형태로 저장
def write_lines(path, lines):
  with open(path, 'w', encoding='utf-8') as f:
    for line in lines:
      f.write(f'{line}\n')

write_lines('/root/train.txt', nsmc.train.get_all_texts())
write_lines('/root/test.txt', nsmc.test.get_all_texts())

In [ ]:
#GPT tokenizer
#한글은 한 글자가 3개의 유니코드 바이트로 표현됨
#BPE를 사용할 것이므로, 한글을 유니코드 바이트로 변환 뒤, 토큰화 수행

#바이트 수준 BPE어휘 집합 구축 결과를 저장해둘 디렉토리 생성
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok=True)

In [ ]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/root/train.txt", "/root/test.txt"], #학습 말뭉치를 list로 제공
    vocab_size=10000, 
    special_tokens=["[PAD]"] #특수 토큰 추가 (mask, bos, eos 등을 관리)
)
bytebpe_tokenizer.save_model("/gdrive/My Drive/nlpbook/bbpe")

['/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/gdrive/My Drive/nlpbook/bbpe/merges.txt']

+ 수행 후에 /gdrive/My Drive/nlpbook/bbpe 디렉토리에 vocab.json, merges.txt가 생성됨
+ vocab.json : 바이트 수준의 BPE 어휘 집합
+ merges.txt : 바이그램 쌍의 병합 우선 순위

## GPT 입력값 만들기

In [ ]:
#GPT tokenizer 선언
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file /gdrive/My Drive/nlpbook/bbpe/config.json not found


In [ ]:
#위에서 불러온 토크나이저로 예시 문장 토큰화
sentences = [
             "안녕 나는 한강 커피스미스에서 와이파이 연결이 안되는 사람이야",
             "나의 미래는 어디로 흘러가는 걸까",
             "단추 단테 귀여워",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [ ]:
#바이트 유니코드 대로 토큰화됨
tokenized_sentences

[['ìķĪë',
  'ħķ',
  'ĠëĤĺëĬĶ',
  'Ġíķľ',
  'ê°ķ',
  'Ġì»¤',
  'íĶ¼ìĬ¤',
  'ë¯¸ìĬ¤',
  'ìĹĲìĦľ',
  'ĠìĻĢ',
  'ìĿ´',
  'íĮĮìĿ´',
  'ĠìĹ°ê²°',
  'ìĿ´',
  'ĠìķĪëĲĺëĬĶ',
  'ĠìĤ¬ëŀĮ',
  'ìĿ´ìķ¼'],
 ['ëĤĺìĿĺ', 'Ġë¯¸ëŀĺ', 'ëĬĶ', 'Ġìĸ´ëĶĶë¡ľ', 'ĠíĿĺëŁ¬ê°ĢëĬĶ', 'Ġê±¸ê¹Į'],
 ['ëĭ¨', 'ì¶Ķ', 'Ġëĭ¨', 'íħĮ', 'Ġê·ĢìĹ¬ìĽĮ']]

In [ ]:
# 실제 모델의 입력으로 사용하는 경우
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length", #문장의 최대 길이에 맞춰 패딩
    max_length=12, #문장 토큰 기준 최대 길이
    truncation=True, #문장 잘림 허용 옵션 허용
)



In [ ]:
#위 코드 결과로 input_ids 와 attention_mask가 만들어진다
#input_ids : 토큰화 결과를 사용하여, 각 토큰을 index로 변환한 것. PAD 토큰은 0으로 indexing 되어있음. 
batch_inputs["input_ids"]
#결과를 보면, 2,3번째 문장은 max_length보다 짧기에 남은 부분이 PAD토큰 즉 0으로 Indexing 되어있음.

[[1226, 8347, 2057, 467, 1174, 2259, 6316, 5972, 462, 1604, 264, 2741],
 [4060, 3835, 270, 7877, 7749, 8645, 0, 0, 0, 0, 0, 0],
 [703, 891, 1196, 1111, 6998, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
#attention_mask은 일반 토큰과 패딩 토큰을 구분해준다. 일반 토큰은 1, 패딩 토큰은 0이다.
batch_inputs["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]